<h1>Problema de Transporte<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

Primero, iniciamos con la llamada del paquete PULP:

In [1]:
from pulp import *

Definimos en dos listas los nodos en nuestro problema de transporte:

In [2]:
#Plantas de producción
Plantas=["x1","x2","x3"]
#Centros de distribución (Llegada)
Centros=["1","2","3","4"]

Definimos, la producción de cada planta:

In [3]:
produccion={"x1":5000,"x2":6000,"x3":2500}

Ahora, definimos la demanda de cada centro de distribución:

In [4]:
demanda={"1":6000,"2":4000,"3":2000,"4":1500}

En una lista de listas incluimos los costos del problema:

In [5]:
costos=[ 
    [3,2,7,6],
    [7,5,2,3],
    [2,5,4,5]
]
costos=makeDict([Plantas,Centros],costos,0)
costos

defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
            {'x1': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'1': 3, '2': 2, '3': 7, '4': 6}),
             'x2': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'1': 7, '2': 5, '3': 2, '4': 3}),
             'x3': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'1': 2, '2': 5, '3': 4, '4': 5})})

Ahora definimos el problema de Programación lineal asociado a este problema de transporte:

In [6]:
probFoster=LpProblem("Problema_de_Foster",LpMinimize)

In [7]:
Rutas = [(p,c) for p in Plantas for c in Centros]

In [8]:
Rutas

[('x1', '1'),
 ('x1', '2'),
 ('x1', '3'),
 ('x1', '4'),
 ('x2', '1'),
 ('x2', '2'),
 ('x2', '3'),
 ('x2', '4'),
 ('x3', '1'),
 ('x3', '2'),
 ('x3', '3'),
 ('x3', '4')]

Usando lo anterior, definimos las variables:

In [9]:
variables=LpVariable.dicts("Ruta",(Plantas,Centros),0,None,LpContinuous)

In [10]:
variables

{'x1': {'1': Ruta_x1_1, '2': Ruta_x1_2, '3': Ruta_x1_3, '4': Ruta_x1_4},
 'x2': {'1': Ruta_x2_1, '2': Ruta_x2_2, '3': Ruta_x2_3, '4': Ruta_x2_4},
 'x3': {'1': Ruta_x3_1, '2': Ruta_x3_2, '3': Ruta_x3_3, '4': Ruta_x3_4}}

Describimos la función objetivo a partir de un for:

In [11]:
probFoster+= lpSum([variables[p][c]*costos[p][c] for (p,c) in Rutas])

In [12]:
for p in Plantas:
    probFoster +=lpSum([variables[p][c] for c in Centros])<=produccion[p],"Producción_en_%s"%p

Restricciones de acuerdo a demanda en los centros:

In [13]:
for c in Centros:
    probFoster +=lpSum([variables[p][c] for p in Plantas])>=demanda[c],"Demanda_en_%s"%c

In [14]:
probFoster.writeLP

<bound method LpProblem.writeLP of Problema_de_Foster:
MINIMIZE
3*Ruta_x1_1 + 2*Ruta_x1_2 + 7*Ruta_x1_3 + 6*Ruta_x1_4 + 7*Ruta_x2_1 + 5*Ruta_x2_2 + 2*Ruta_x2_3 + 3*Ruta_x2_4 + 2*Ruta_x3_1 + 5*Ruta_x3_2 + 4*Ruta_x3_3 + 5*Ruta_x3_4 + 0
SUBJECT TO
Producción_en_x1: Ruta_x1_1 + Ruta_x1_2 + Ruta_x1_3 + Ruta_x1_4 <= 5000

Producción_en_x2: Ruta_x2_1 + Ruta_x2_2 + Ruta_x2_3 + Ruta_x2_4 <= 6000

Producción_en_x3: Ruta_x3_1 + Ruta_x3_2 + Ruta_x3_3 + Ruta_x3_4 <= 2500

Demanda_en_1: Ruta_x1_1 + Ruta_x2_1 + Ruta_x3_1 >= 6000

Demanda_en_2: Ruta_x1_2 + Ruta_x2_2 + Ruta_x3_2 >= 4000

Demanda_en_3: Ruta_x1_3 + Ruta_x2_3 + Ruta_x3_3 >= 2000

Demanda_en_4: Ruta_x1_4 + Ruta_x2_4 + Ruta_x3_4 >= 1500

VARIABLES
Ruta_x1_1 Continuous
Ruta_x1_2 Continuous
Ruta_x1_3 Continuous
Ruta_x1_4 Continuous
Ruta_x2_1 Continuous
Ruta_x2_2 Continuous
Ruta_x2_3 Continuous
Ruta_x2_4 Continuous
Ruta_x3_1 Continuous
Ruta_x3_2 Continuous
Ruta_x3_3 Continuous
Ruta_x3_4 Continuous
>

In [15]:
probFoster.solve()

1

In [16]:
for v in probFoster.variables():
    print(v.name, "=", v.varValue)

Ruta_x1_1 = 3500.0
Ruta_x1_2 = 1500.0
Ruta_x1_3 = 0.0
Ruta_x1_4 = 0.0
Ruta_x2_1 = 0.0
Ruta_x2_2 = 2500.0
Ruta_x2_3 = 2000.0
Ruta_x2_4 = 1500.0
Ruta_x3_1 = 2500.0
Ruta_x3_2 = 0.0
Ruta_x3_3 = 0.0
Ruta_x3_4 = 0.0


In [17]:
print("Costominimo= ", value(probFoster.objective))

Costominimo=  39500.0
